In [20]:
%load_ext autoreload
%autoreload 2

import os

from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

from adni_2d import ADNIDataset2D
import torchvision.transforms as transforms
from dataset_train2D import RandomGenerator

# Training settings
batch_size = 32
iteration = 10000
lr = [0.001, 0.01]
momentum = 0.9
cuda = True
seed = 8
log_interval = 20
l2_decay = 5e-4
source1_name = "GE"
source2_name = "Philips"
target_name = "Siemens"
dataset = "ADNI1"

IMG_PATH = "./Dataset/ADNI1"
results_dir = "./Results"
img_size = 224
num_workers = 4

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
dataset = ADNIDataset2D(
    root="data\\preprocess\\ADNI1_T1_MRI\\6_step_nifti_2d",
    domain=source1_name,
    transform=None,
)

2024-08-06 20:39:58.603 | INFO     | adni_2d:__init__:18 - +++++++++++++++Loading Image and Label data from ADNI1++++++++++++++++
2024-08-06 20:39:58.638 | INFO     | adni_2d:__init__:22 - ++++++++++++++++++++++++++Loading Label Data++++++++++++++++++++++++++


In [22]:
len(dataset)

10854

In [23]:
# Get class names as a list
class_names = dataset.classes
class_names

['AD', 'CN']

In [24]:
# Can also get class names as a dict
class_dict = dataset.class_to_idx
class_dict

{'AD': 0, 'CN': 1}

In [25]:
img, label = dataset[0][0], dataset[0][1]
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

KeyError: 0

In [11]:
# Rearrange the order of dimensions
img_permute = img.permute(1, 2, 0)

# Print out different shapes (before and after permute)
print(f"Original shape: {img.shape} -> [color_channels, height, width]")
print(f"Image permute shape: {img_permute.shape} -> [height, width, color_channels]")

# Plot the image
plt.figure(figsize=(10, 7))
plt.imshow(img.permute(1, 2, 0))
plt.axis("off")
plt.title(class_names[label], fontsize=14)

NameError: name 'img' is not defined